In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url #store the URL
        response = requests.get(url, headers=HEADERS) #Make a request to the URL
        soup = BeautifulSoup(response.content, 'html.parser') #Parse the HTML
        self.title = soup.title.string if soup.title else "No title found" #Extract the Title
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [4]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [5]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [6]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [7]:
def summarize(url):
    website = Website(url)
    response = requests.post(OLLAMA_API, json={
        "model": MODEL,
        "messages": messages_for(website),
        "stream": False
    }, headers=HEADERS)
    result = response.json()
    return display(Markdown(result["message"]["content"]))

In [8]:
summarize("https://www.toyota.lk")

**Summary of Toyota Lanka**
=====================================

### Overview
-----------------

Toyota Lanka is an authorized distributor of original Pirelli tires, genuine Motor Oil, and new Toyota vehicles.

### Key Features
--------------

* Offers a wide range of Toyota models, including Compact SUVs, Hatches, MPVs, Pickups, Buses, Sedans, and SUVs.
* Provides Genuine Toyota Motor Oil for passenger cars.
* Has a commitment to delivering world-renowned "Toyota Quality Service".
* Has authorized dealerships across Sri Lanka.

### News/Announcements
-----------------------

* **Road Trips into the Rough**: A documentary series showcasing Sri Lanka's natural wonders.
* **Schedule Your Test Drive Today**: Book a test drive with Toyota Lanka to experience their models firsthand.

### Latest Promotions
----------------------

* Various promotions and offers on select models and services.

### Contact Information
---------------------

* Phone: +94 11 293 9000 (ext. -6)
* Fax: +94 11 293 9005
* Email: [info@toyota.lk](mailto:info@toyota.lk)

### Disclaimer
----------------

Vehicle body color may differ from printed photos on the website and catalog.
Toyota Motor Corporation reserves the right to alter specifications and equipment without notice.